In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import gc

# Step 1: Load the Data
train_identity = pd.read_csv(r'/content/train_identity.csv')
train_transaction = pd.read_csv(r"/content/train_transaction.csv")
test_identity = pd.read_csv(r"/content/test_identity.csv")
test_transaction = pd.read_csv(r"/content/test_transaction.csv")
sample_submission = pd.read_csv(r"/content/sample_submission.csv")

# Step 1.1: Fix column names in test_identity
test_identity.columns = [col.replace('id-', 'id_') for col in test_identity.columns]

# Step 1.2: Combine the datasets
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

# Step 1.3: Optimize memory by downcasting numerical columns
def downcast_df(df):
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    return df

train = downcast_df(train)
test = downcast_df(test)

# Store TransactionID for test set alignment
test_ids = test['TransactionID'].copy()

# Clean up memory
del train_identity, train_transaction, test_identity, test_transaction
gc.collect()

print(f"Train shape: {train.shape}, Test shape: {test.shape}")

# Step 2: Preprocessing
# 2.1: Remove features with high missing values (>60%)
missing_percent = train.isnull().mean()
high_missing_cols = missing_percent[missing_percent > 0.6].index.tolist()
if 'TransactionID' in high_missing_cols:
    high_missing_cols.remove('TransactionID')
train.drop(columns=high_missing_cols, inplace=True)
test.drop(columns=high_missing_cols, inplace=True)

# 2.2: Define categorical columns
categorical_cols = ['ProductCD', 'card4', 'card5', 'card6']
categorical_cols = [col for col in categorical_cols if col in train.columns]

# 2.3: Handle missing values
numerical_cols = train.select_dtypes(include=['float32', 'float64', 'int8', 'int16', 'int32']).columns.tolist()
if 'isFraud' in numerical_cols:
    numerical_cols.remove('isFraud')
if 'TransactionID' in numerical_cols:
    numerical_cols.remove('TransactionID')

for col in numerical_cols:
    train[col] = train[col].fillna(train[col].median())
    test[col] = test[col].fillna(test[col].median())

for col in categorical_cols:
    train[col] = train[col].fillna('missing').astype(str)
    test[col] = test[col].fillna('missing').astype(str)

# 2.4: Encode categorical features early
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    combined = pd.concat([train[col], test[col]], axis=0)
    le.fit(combined)
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])
    label_encoders[col] = le

# Step 3: Feature Engineering
# 3.1: Time-based feature
train['hour'] = (train['TransactionDT'] // 3600) % 24
test['hour'] = (test['TransactionDT'] // 3600) % 24

# 3.2: Log transform TransactionAmt
train['LogTransactionAmt'] = np.log1p(train['TransactionAmt'])
test['LogTransactionAmt'] = np.log1p(test['TransactionAmt'])

# 3.3: Transaction frequency for card1
if 'card1' in train.columns:
    freq_map = train['card1'].value_counts().to_dict()
    train['card1_freq'] = train['card1'].map(freq_map)
    test['card1_freq'] = test['card1'].map(freq_map)
    train['card1_freq'] = train['card1_freq'].fillna(0)
    test['card1_freq'] = test['card1_freq'].fillna(0)

# Step 4: Feature Preparation for SVM
# 4.1: Define feature set
features = ['LogTransactionAmt', 'hour', 'card1_freq'] + categorical_cols + \
           [col for col in ['C1', 'C2', 'C3', 'C4', 'D1', 'D2', 'D3'] if col in train.columns]

# Ensure all features exist
features = [f for f in features if f in train.columns]

X = train[features]
y = train['isFraud']
X_test_full = test[features]

# 4.2: Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test_full)

# 4.3: SMOTE with full oversampling
smote = SMOTE(random_state=42, sampling_strategy=1.0)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# 4.4: Create train/val/test split (70/15/15)
X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 5: Train SVM Model
svm = SVC(kernel='rbf',
          class_weight='balanced',
          probability=True,
          random_state=42,
          C=3.0,
          gamma='scale',
          max_iter=3000)

svm.fit(X_train, y_train)

# Step 6: Evaluate on Test Set (no threshold optimization)
test_probs = svm.predict_proba(X_test)[:, 1]
test_pred = (test_probs >= 0.5).astype(int)

test_precision = precision_score(y_test, test_pred, zero_division=0)
test_recall = recall_score(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred)
test_auc = roc_auc_score(y_test, test_probs)
test_accuracy = accuracy_score(y_test, test_pred)

print("\nTest Set Performance:")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1: {test_f1:.4f}")
print(f"Test AUC: {test_auc:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Step 7: Evaluate on Train Set
train_probs = svm.predict_proba(X_train)[:, 1]
train_pred = (train_probs >= 0.5).astype(int)

train_precision = precision_score(y_train, train_pred, zero_division=0)
train_recall = recall_score(y_train, train_pred)
train_f1 = f1_score(y_train, train_pred)
train_auc = roc_auc_score(y_train, train_probs)
train_accuracy = accuracy_score(y_train, train_pred)

print("\nTrain Set Performance:")
print(f"Train Precision: {train_precision:.4f}")
print(f"Train Recall: {train_recall:.4f}")
print(f"Train F1: {train_f1:.4f}")
print(f"Train AUC: {train_auc:.4f}")
print(f"Train Accuracy: {train_accuracy:.4f}")

# Step 8: Generate Predictions for Submission
test_full_probs = svm.predict_proba(X_test_scaled)[:, 1]

# Create a DataFrame with TransactionID and predictions
test_pred_df = pd.DataFrame({
    'TransactionID': test_ids,
    'isFraud': test_full_probs
})

# Merge with sample_submission to align predictions
sample_submission = sample_submission.merge(test_pred_df, on='TransactionID', how='left', suffixes=('', '_pred'))
sample_submission['isFraud'] = sample_submission['isFraud_pred'].fillna(test_full_probs.mean())
sample_submission = sample_submission[['TransactionID', 'isFraud']]

# Save submission
sample_submission.to_csv('submission.csv', index=False)
print("\nSubmission file 'submission.csv' created successfully!")

Train shape: (9875, 434), Test shape: (11069, 433)


<ipython-input-12-3557517822>:84: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['hour'] = (train['TransactionDT'] // 3600) % 24
<ipython-input-12-3557517822>:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test['hour'] = (test['TransactionDT'] // 3600) % 24
<ipython-input-12-3557517822>:88: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm


Test Set Performance:
Test Precision: 0.8652
Test Recall: 0.9001
Test F1: 0.8823
Test AUC: 0.9520
Test Accuracy: 0.8800

Train Set Performance:
Train Precision: 0.8797
Train Recall: 0.9032
Train F1: 0.8913
Train AUC: 0.9547
Train Accuracy: 0.8898

Submission file 'submission.csv' created successfully!
